# Lab 2

## Objectives

To install custom firmware on the controller unit, aquire time-sampled readings by sending command strings, and learn to visualize the data

### Communication Channel with device

In [1]:
# communication channel setup 
import serial 
import serial.tools.list_ports 
 
VID = 61525 
PID = 38912 
 
ports = serial.tools.list_ports.comports() 
 
device = None 
for p in ports: 
    if p.vid == VID and p.pid == PID: 
        try:  
            device = serial.Serial(p.device) 
        except serial.SerialException: 
            print('Reconnect the controller unit.') 
 
if device is None: 
    raise Exception('No suitable device detected.')

Exception: No suitable device detected.

### Setting VDCP

#### Command string

Format: 'dzAAAA\r'

-  'dz' instructs the controller unit to set the DC supply voltage. 


-  'AAAA' represents the intended DC voltage value in **10 mV** increments, with zero-padding to fill up four-character spaces. 


-  '\r' is the carriage return character

#### Setting VDCP to 8V

In [3]:
vdc = 8
cmd_setvdc = 'dz' + str(int(vdc*100)).zfill(4) + '\r'

'int(vdc*100)' - converts the desired voltage balue to an integer by multiplying by 100. 

'str().zfill()' - converts the above integer value into a string and adds '0' at the front until the string reaches the specified length indicated in the bracket.

#### Setting VDCP to 12V

In [ ]:
vdc = 12
cmd_setvdc = 'dz' + str(int(vdc*100)).zfill(4) + '\r'

##### Sending the command to the controller unit

In [2]:
device.write(bytes(cmd_setvdc, 'utf-8'))

SyntaxError: incomplete input (3078679042.py, line 1)

### Generating timed voltage singnals

#### Command string

Format: 'sBCCDDDEEEEEEEFFFFGGGG\r'
-  'sB' indicates the channel for generating the waveform with '1' or '2' replacing 'B' to represent W1 or W2, respectively. 


-  'CC' denotes the waveform shape. Currently, three shapes are supported: sinusoidal ('00'), triangular ('10'), and sawtooth ('11'). 


-  'DDD' specifies the number of samples per oscillation cycle. 


-  'EEEEEEE' represents the oscillation frequency in Hz. 


-  'FFFF' is the signal amplitude in 10 mV. 


-  'GGGG' is the DC offset in 10 mV.

#### Creating and measuring a sinusoidal waveform 
##### 2kHz freq, 0.5V Amplitude, 0V DC offset on W1

In [4]:
ns = 64             # samples per cycle; 64 generally enough 
freq = 2000         # frequency in Hz
amp = 0.5           # amplitude 
offset = 0          # DC offset 
cmd_gensin = 's100' # W1 sinusoidal 
cmd_gensin += str(ns).zfill(3) + str(freq).zfill(7) + str(int(amp*100)).zfill(4) + str(int(offset*100)).zfill(4) + '\r' 
device.write(bytes(cmd_gensin, 'utf-8'))    #sending the command string to controller unit

AttributeError: 'NoneType' object has no attribute 'write'

##### Measurements

Vp = 

Vrms(ideal) = 0.707Vp

Vrms(measured) = 

Freq = 

#### Creating and measuring a triangular waveform 
##### 2kHz freq, 0.5V Amplitude, 0V DC offset on W1

In [5]:
ns = 64             # samples per cycle; 64 generally enough 
freq = 2000         # frequency in Hz
amp = 0.5           # amplitude 
offset = 0          # DC offset 
cmd_gensin = 's110' # W1 triangular
cmd_gensin += str(ns).zfill(3) + str(freq).zfill(7) + str(int(amp*100)).zfill(4) + str(int(offset*100)).zfill(4) + '\r' 
device.write(bytes(cmd_gensin, 'utf-8'))    #sending the command string to controller unit

AttributeError: 'NoneType' object has no attribute 'write'

##### Measurements

Vp = 

Vrms(ideal) = 0.707Vp

Vrms(measured) = 

Freq = 

#### Discussion of the change of meaurements

### Reading timed voltage signals

#### Command String

Format: 'm1HHHHHHIJJJKKKLMMMNNN\r'
-  'm1', instructs the controller unit to read timed voltage signals from CH1 and CH2.


-  'HHHHHH' represents the sampling frequency given in Hz. For our current implementation of the controller unit, the maximum sampling rate that can reliably read both channels is 210kHz.


-  'IJJJKKK' and 'LMMMNNN' are the coupling mode ('I' and 'L'), gain ('JJJ' and 'MMM'), and DC offset ('KKK' and 'NNN') parameters for CH1 and CH2, respectively. These values are 
crucial for configuring the ADC to accurately measure the signal within the expected voltage range. Calibration is necessary to determine the values of these parameters for different voltage ranges. In particular, the DC bias values can vary between units and need to be adjusted individually.